In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras import layers
from keras import models
from keras import regularizers
import joblib
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.callbacks import EarlyStopping

In [2]:
DATASET_PATH = 'dataset_020/color_dataset_concat.csv'
MODEL_PATH = os.path.join('MODEL', 'color_model.keras')
SCALER_PATH = os.path.join('MODEL', 'color_scaler.pkl')

In [3]:
# Load the dataset
df = pd.read_csv(DATASET_PATH)

X = df[['R', 'G', 'B', 'W']]
y = df['control_value']

In [4]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, SCALER_PATH)

['MODEL\\color_scaler.pkl']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [6]:
model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=(4,), 
                     kernel_regularizer=regularizers.l2(0.01)),  # L2 Regularization
        layers.Dropout(0.3),  # Dropout for regularization
        layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.3),
        layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(1, activation='linear')  # Linear activation for output
    ])
    
model.compile(optimizer='adam', 
                loss='mean_squared_error',
                metrics=['mean_absolute_error', 'mean_squared_error'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11

In [7]:
# Create early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=200, verbose=1, batch_size=1, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/200
15968/15968 [==============================] - 21s 1ms/step - loss: 0.0953 - mean_absolute_error: 0.2512 - mean_squared_error: 0.0846 - val_loss: 0.0821 - val_mean_absolute_error: 0.2478 - val_mean_squared_error: 0.0821
Epoch 2/200
15968/15968 [==============================] - 22s 1ms/step - loss: 0.0840 - mean_absolute_error: 0.2510 - mean_squared_error: 0.0840 - val_loss: 0.0823 - val_mean_absolute_error: 0.2478 - val_mean_squared_error: 0.0823
Epoch 3/200
15934/15968 [============================>.] - ETA: 0s - loss: 0.0841 - mean_absolute_error: 0.2510 - mean_squared_error: 0.0841

In [ ]:
# Evaluate the model
loss, mae, mse = model.evaluate(X_test, y_test, verbose=2)

In [ ]:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('Model MAE')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model.save(MODEL_PATH)

In [ ]:
# Use the model for prediction
predictions = model.predict(X_test)

In [ ]:
# Plot the predictions against the actual values
sns.scatterplot(x=y_test, y=predictions.flatten())
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.show()


In [ ]:
X_train.shape[1]

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Reset the index of y_test to allow direct indexing after the train-test split
y_test = y_test.reset_index(drop=True)

# Calculate prediction errors
errors = predictions.flatten() - y_test

# Get the indices of the best and worst predictions
best_pred_indices = np.argsort(np.abs(errors))[:10]  # Indices of the 10 best predictions
worst_pred_indices = np.argsort(np.abs(errors))[-100:]  # Indices of the 10 worst predictions



absolute_errors = np.abs(errors)

best_pred_errors = absolute_errors.iloc[best_pred_indices]
worst_pred_errors = absolute_errors.iloc[worst_pred_indices]

print("Best Prediction Errors:")
for i, index in enumerate(best_pred_indices):
    print(f"Index: {index}, Absolute Error: {best_pred_errors.iloc[i]}")

print("\nWorst Prediction Errors:")
for i, index in enumerate(worst_pred_indices):
    print(f"Index: {index}, Absolute Error: {worst_pred_errors.iloc[i]}")
    



# Adjust your slicing here, if necessary
slice_min = 0
slice_max = len(y_test)  
col_slice = 3  # The feature  to plot


sns.set_style("whitegrid")
plt.figure(figsize=(20,12))
plt.title("Predicted vs Actual", fontsize=20)

sns.scatterplot(x=y_test[slice_min:slice_max], y=X_test[slice_min:slice_max, col_slice], color="blue", label="Actual", linewidth=1, alpha=0.8)
sns.scatterplot(x=predictions.flatten()[slice_min:slice_max], y=X_test[slice_min:slice_max, col_slice], color="orange", label="Predicted", linewidth=1, alpha=0.8)

# Highlight the best predictions
sns.scatterplot(x=y_test[best_pred_indices], y=X_test[best_pred_indices, col_slice], color="green", label="Best Predictions", s=100, edgecolor='black')

# Highlight the worst predictions
sns.scatterplot(x=y_test[worst_pred_indices], y=X_test[worst_pred_indices, col_slice], color="red", label="Worst Predictions", s=100, edgecolor='black')


plt.xlabel('Target Control Value', color='black', fontsize=20)
plt.ylabel('Sensor Light', color='black', fontsize=20)
plt.legend(loc='upper right', fontsize=20)
plt.show()
